In [1]:
# imprt the dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



In [2]:
city_data_df = pd.read_csv("Resources/WeatherPy_challenge.csv")


In [3]:
# Filter the DataFrame for minimum and maximum temperature preferences
columns = city_data_df.columns.tolist()
columns

['Unnamed: 0',
 'City',
 'Country',
 'Date',
 'Lat',
 'Lng',
 'Max Temp',
 'Humidity',
 'Cloudiness',
 'Wind Speed',
 'Current Description',
 'Rain inches (last 3hours)',
 'Snow inches(last 3hours)']

In [4]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = input("Do you want it to be raining? (yes/no) ")
snow = input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip?  90
What is the maximum temperature you would like for your trip?  75
Do you want it to be raining? (yes/no)  no
Do you want it to be snowing? (yes/no)  no


In [5]:
for index, row in city_data_df.iterrows():
    if rain == "no":
        rain_inches = city_data_df['Rain inches (last 3hours)'] == 0
    else:
        rain_inches = city_data_df['Rain inches (last 3hours)'] != 0
    if snow == "no":
        snow_inches = city_data_df['Snow inches(last 3hours)'] == 0
    else:
        snow_inches = city_data_df['Snow inches(last 3hours)'] != 0
        



In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    
preferred_cities_df = city_data_df.loc[(city_data_df['Rain inches (last 3hours)'] != rain_inches) &
                                       (city_data_df['Snow inches(last 3hours)'] != snow_inches) &
                                       (city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                       
preferred_cities_df.head()

,Unnamed: 0,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3hours),Snow inches(last 3hours)
26,26,Navrongo,GH,2020-04-02 13:28:28,10.90,-1.09,102.33,21,57,3.20,broken clouds,0.0,0.0
96,96,Taoudenni,ML,2020-04-02 13:28:37,22.68,-3.98,98.46,8,97,8.88,overcast clouds,0.0,0.0
140,140,Arlit,NE,2020-04-02 13:28:43,18.74,7.39,102.52,6,9,7.99,clear sky,0.0,0.0
144,144,Beboto,TD,2020-04-02 13:28:43,8.27,16.94,114.21,12,80,2.68,broken clouds,0.0,0.0
173,173,Shahada,IN,2020-04-02 13:28:48,21.55,74.47,102.47,11,0,9.46,clear sky,0.0,0.0


In [7]:
# Add the cities to a marker layer map with a pop-up marker for each city that includes

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
26,Navrongo,GH,102.33,broken clouds,10.90,-1.09,
96,Taoudenni,ML,98.46,overcast clouds,22.68,-3.98,
140,Arlit,NE,102.52,clear sky,18.74,7.39,
144,Beboto,TD,114.21,broken clouds,8.27,16.94,
173,Shahada,IN,102.47,clear sky,21.55,74.47,


In [8]:
import requests
# Import the API key.
from config1 import g_key



In [9]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [10]:
len(hotels["results"])

20

In [11]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
import gmaps
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [13]:
# Get the maximum temperature.
max_temp = hotel_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [14]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Heatmap of temperature
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [16]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
26,Navrongo,GH,102.33,broken clouds,10.90,-1.09,Azakka Hotel
96,Taoudenni,ML,98.46,overcast clouds,22.68,-3.98,
140,Arlit,NE,102.52,clear sky,18.74,7.39,Hôtel Telwa Bungalow
144,Beboto,TD,114.21,broken clouds,8.27,16.94,
173,Shahada,IN,102.47,clear sky,21.55,74.47,AVADHOOT LODGE


In [17]:
hotel_df.to_csv("Resources/WeatherPy_vacation.csv")